# Project: Marvel Cinematic Universe's Acquintance network


This project is about the MCU and its acquintance network created by different actors / actresses. This means that we have to find all connections, weight the network accordingly and give different measured parameters of it.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

import sys
import os
import glob
import pathlib

In [4]:
mcu_orig = pd.read_csv("data/characters.csv")
display(mcu_orig.head())
display(mcu_orig.info())

,character,Ant-Man,Avengers: Age of Ultron,Avengers: Endgame,Avengers: Infinity War,Black Panther,Captain America: Civil War,Captain America: The First Avenger,Captain America: The Winter Soldier,Captain Marvel,...,Spider-Man: Homecoming,The Avengers,Thor,Thor: Ragnarok,Thor: The Dark World,total lines,total words,movie appearances,avg words per line,avg lines per movie
0,TONY STARK,0,173,137,119,0,165,0,0,0,...,74,152,0,0,0,1788,22590,9,12,198
1,STEVE ROGERS,3,133,119,25,0,154,186,218,3,...,7,121,0,0,0,977,8653,10,8,97
2,THOR,0,55,70,74,0,0,0,0,0,...,0,47,166,289,133,863,9487,7,10,123
3,PETER PARKER,0,0,7,39,0,73,0,0,0,...,402,0,0,0,0,551,5517,4,10,137
4,NATASHA ROMANOFF,1,79,69,15,0,64,0,112,3,...,0,97,0,1,0,498,4689,10,9,49


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 652 entries, 0 to 651
Data columns (total 24 columns):
 #   Column                               Non-Null Count  Dtype 
---  ------                               --------------  ----- 
 0   character                            652 non-null    object
 1   Ant-Man                              652 non-null    int64 
 2   Avengers: Age of Ultron              652 non-null    int64 
 3   Avengers: Endgame                    652 non-null    int64 
 4   Avengers: Infinity War               652 non-null    int64 
 5   Black Panther                        652 non-null    int64 
 6   Captain America: Civil War           652 non-null    int64 
 7   Captain America: The First Avenger   652 non-null    int64 
 8   Captain America: The Winter Soldier  652 non-null    int64 
 9   Captain Marvel                       652 non-null    int64 
 10  Guardians of the Galaxy Vol. 2       652 non-null    int64 
 11  Iron Man                             652 non-

None

In [5]:
display(mcu_orig['character'].values)

array(['TONY STARK', 'STEVE ROGERS', 'THOR', 'PETER PARKER',
       'NATASHA ROMANOFF', 'PEPPER POTTS', 'BRUCE BANNER', 'NICK FURY',
       'LOKI', 'SCOTT LANG', 'JAMES RHODES', 'PETER QUILL',
       'CAROL DANVERS', 'JANE FOSTER', 'ROCKET', 'CLINT BARTON',
       'SAM WILSON', 'GAMORA', "T'CHALLA", 'HAPPY HOGAN', 'HANK PYM',
       'ODIN', 'JARVIS', 'NED LEEDS', 'ERIK SELVIG', 'PHIL COULSON',
       'BUCKY BARNES', 'THANOS', 'DRAX', 'WANDA MAXIMOFF', 'DARCY LEWIS',
       'ADRIAN TOOMES', 'PEGGY CARTER', 'SHURI', 'HOPE VAN DYNE',
       'VALKYRIE', 'LUIS', 'JUSTIN HAMMER', 'ULTRON', 'MARIA HILL',
       'NEBULA', 'STEPHEN STRANGE', 'VISION', 'VOLSTAGG', 'NAKIA',
       'LIZ TOOMES', 'YON-ROGG', 'YONDU', 'FRIGGA', 'OKOYE', 'KAREN',
       'MANTIS', 'EVERETT ROSS', 'MAYA HANSEN', 'HOWARD STARK', 'SIF',
       'ALEXANDER PIERCE', 'COL. CHESTER PHILLIPS', 'TALOS',
       'ALDRICH KILLIAN', 'EGO', 'FANDRAL', 'RED SKULL', 'DARREN CROSS',
       "N'JADAKA (KILLMONGER)", 'HEIMDALL', 'HARLEY K